In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"data\processed_df_for_consort.csv")

In [3]:
df.columns

Index(['clean_id', 'raw_id', 'clinic_application_date', 'intake_date', 'group',
       'signing_date', 'sheet', 'therapy_start_date', 'therapy_end_date',
       'has_source_CAU', 'has_source_IPC-SSC', 'has_source_אי הסכמה למחקר',
       'has_source_אי התאמה למחקר', 'has_source_אין שת"פ טיפולי',
       'has_source_משתתפים פעילים', 'has_source_נשירה מחקרית',
       'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_סיימו טיפול',
       'has_source_עלייה לרמה 2', 'has_source_פספוסי גיוסים',
       'first_contact_date', 'therapy_starting_date'],
      dtype='object')

In [4]:
exclude = ['has_source_נשירה מחקרית', 'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_אין שת"פ טיפולי', 'has_source_סיימו טיפול', 'has_source_אי הסכמה למחקר', 'has_source_אי התאמה למחקר', 'has_source_פספוסי גיוסים']

In [5]:
N = {
    'isin':['has_source_CAU', 'has_source_IPC-SSC', 'has_source_אי הסכמה למחקר',
       'has_source_אי התאמה למחקר', 'has_source_אין שת"פ טיפולי',
       'has_source_משתתפים פעילים', 'has_source_נשירה מחקרית',
       'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_סיימו טיפול',
       'has_source_עלייה לרמה 2', 'has_source_פספוסי גיוסים'],
    'not_in': []
}

Eligible = {
            'isin':['has_source_CAU', 'has_source_IPC-SSC', 'has_source_אי הסכמה למחקר',
                   'has_source_אין שת"פ טיפולי', 
                   'has_source_משתתפים פעילים', 'has_source_נשירה מחקרית',
                   'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_סיימו טיפול',
                   'has_source_עלייה לרמה 2'],
            'not_in': []
}


Randomized = {
            'isin':['has_source_CAU', 'has_source_IPC-SSC',
           'has_source_אין שת"פ טיפולי',
           'has_source_משתתפים פעילים', 'has_source_נשירה מחקרית',
           'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_סיימו טיפול',
           'has_source_עלייה לרמה 2'],
           'not_in': []
}


Dropout = {
           'isin':['has_source_נשירה מחקרית', 'has_source_נשירה קלינית- לאחר ת. טיפול'],
            'not_in': []
                    }


research_dropout = {
                    'isin':['has_source_נשירה מחקרית'],
                    'not_in': []
                    }

clinical_dropout = {
                    'isin':['has_source_נשירה קלינית- לאחר ת. טיפול'],
                    'not_in': []
                    }


in_waiting_list = {
    'isin': ['has_source_משתתפים פעילים'],
    'not_in': ['has_source_CAU', 'has_source_IPC-SSC'] + exclude
}


Finished = {
    'isin': ['has_source_סיימו טיפול'],
    'not_in': []
}


Active = {
    'isin': ['has_source_משתתפים פעילים', 'has_source_CAU', 'has_source_IPC-SSC'],
    'not_in': exclude
}

not_cooperative = {
    'isin': ['has_source_אין שת"פ טיפולי'],
    'not_in': []
}

In [6]:
consort_groups = {
    
    'N': N,
    'Eligible': Eligible,
    'Randomized': Randomized,
    'Dropout': Dropout,
    'Research Dropout': research_dropout,
    'Clinical Dropout': clinical_dropout,
    'In Waiting List': in_waiting_list,
    'Finished': Finished,
    'Active': Active,
    'Not Cooperative': not_cooperative

}

In [18]:
# Extracting the detials of how to calc the groups 

# rows = []
# for i,j in consort_groups.items():
#     row = {
#         "consort_group": i,
#         "included_tables": j["isin"],
#         "excluded_tables": j["not_in"]
#     }
#     rows.append(row)

# pd.DataFrame(rows).to_csv("consort_groups_definition.csv", index=0)

In [7]:
def isin_consort_group(row, consort_group):
    in_pos_sheet = False
    for sheet in consort_group['isin']:
        if row[sheet]:
            in_pos_sheet = True

    in_neg_sheet = False
    for sheet in consort_group['not_in']:
        if row[sheet]:
            in_neg_sheet = True

    return (in_pos_sheet and (not in_neg_sheet))

In [8]:
for consort_calc in consort_groups.keys():
    df[consort_calc] = df.apply(isin_consort_group, axis=1, args = [consort_groups[consort_calc]])

In [9]:
for i in consort_groups.keys():
    print(i, df[i].sum())

N 301
Eligibility 162
Randomized 129
Dropout 39
Research Dropout 27
Clinical Dropout 12
In Waiting List 38
Finished 30
Active 43
Not Cooperative 17


In [10]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [11]:
df[start] = df[start].replace({'0': pd.NaT})
df[end] = df[end].replace({'0': pd.NaT})


In [12]:
df['waiting_duration'] =  pd.to_datetime(df[end]) - pd.to_datetime(df[start])
df['waiting_duration'] = df['waiting_duration'].dt.days

In [13]:
df['did_started_therapy'] = pd.notna(df[end])

In [26]:
# df.to_csv(r"data\export_with_days_delta.csv", index=0)
df.to_excel(r"data\export_with_days_delta.xlsx", index=0)